In [1]:
import tensorflow as tf
K = tf.keras.backend
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.models import DCN
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names


DeepCTR version 0.9.3 detected. Your version is 0.9.1.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.3


In [11]:
data=pd.read_csv('../data/data_bert.csv',encoding='utf-8')

In [12]:
data.head()

,Unnamed: 0,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,...,tag_bert758,tag_bert759,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767
0,0,207000097,24696418,1.0,5,0,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
1,1,207000097,32507861,1.0,6,2,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
2,2,207000097,39060478,1.0,6,2,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
3,3,207000097,85909646,1.0,6,2,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011
4,4,207000097,172874760,1.0,5,0,27,3914,0.999946,0.999983,...,0.987675,0.924667,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011


In [4]:
data.tail()

,Unnamed: 0,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,...,tag_bert758,tag_bert759,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767
845459,845459,507078823,9873156,0.440,6,0,0,12,0.999862,0.999886,...,0.974894,0.995783,-0.989222,0.999992,0.94585,-0.997905,-0.995735,-0.999130,-0.994474,0.993043
845460,845460,764155292,85274975,0.232,5,0,0,19,0.999687,0.999936,...,0.999685,0.996559,-0.993992,0.999981,0.94837,-0.999431,-0.997548,-0.999439,-0.977021,0.855312
845461,845461,764155292,51463228,0.000,6,0,0,19,0.999687,0.999936,...,0.999685,0.996559,-0.993992,0.999981,0.94837,-0.999431,-0.997548,-0.999439,-0.977021,0.855312
845462,845462,764155292,693496460,0.005,3,0,0,19,0.999687,0.999936,...,0.999685,0.996559,-0.993992,0.999981,0.94837,-0.999431,-0.997548,-0.999439,-0.977021,0.855312
845463,845463,718667726,346747239,0.753,4,0,1,57,0.998748,0.999991,...,0.980760,0.997610,-0.997252,0.999985,0.96784,-0.996740,-0.999278,-0.996762,-0.996212,0.990695


In [13]:
data=data.drop(['Unnamed: 0'],axis=1)

In [14]:
data['user_nm']=0.
data['dance_nm']=0.

In [15]:
from tqdm import tqdm

In [16]:
#user_nm赋值
k = 10000/(data["user_id"].max()-data["user_id"].min())
for i in tqdm(range(len(data))):
    
    transform_value=k*(data["user_id"][i]-data["user_id"].min()) 
    data["user_nm"][i]=transform_value


  0%|                                                                                       | 0/845464 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_10452\3739655352.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["user_nm"][i]=transform_value
100%|████████████████████████████████████████████████████████████████████████| 845464/845464 [2:53:23<00:00, 81.27it/s]


In [17]:
#dance_nm赋值
k1 = 10000/(data["dance_id"].max()-data["dance_id"].min())
for i in tqdm(range(len(data))):
    transform_value=k1*(data["dance_id"][i]-data["dance_id"].min())+1 
#     print(transform_value)
    data["dance_nm"][i]=transform_value

  0%|                                                                                       | 0/845464 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_10452\3916622010.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["dance_nm"][i]=transform_value
100%|████████████████████████████████████████████████████████████████████████| 845464/845464 [2:56:02<00:00, 80.04it/s]


In [19]:
data.to_csv("data_bert_nm.csv",encoding='utf-8')

In [ ]:
cols = data.columns

In [ ]:
cols

In [7]:
len([column for column in data][7:])

1536

In [2]:
data=pd.read_csv("../data/data_bert_nm.csv",encoding='utf-8')

In [3]:
data.head()

,Unnamed: 0,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,...,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767,user_nm,dance_nm
0,0,207000097,24696418,1.0,5,0,27,3914,0.999946,0.999983,...,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011,115.001742,1.0
1,1,207000097,32507861,1.0,6,2,27,3914,0.999946,0.999983,...,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011,151.376687,1.0
2,2,207000097,39060478,1.0,6,2,27,3914,0.999946,0.999983,...,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011,181.889753,1.0
3,3,207000097,85909646,1.0,6,2,27,3914,0.999946,0.999983,...,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011,400.048661,1.0
4,4,207000097,172874760,1.0,5,0,27,3914,0.999946,0.999983,...,-0.986258,0.999988,0.968072,-0.991969,-0.97649,-0.989061,-0.999899,-0.331011,805.012401,1.0


In [4]:
data=data.drop(['Unnamed: 0'],axis=1)

In [5]:
data

,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,title_bert2,...,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767,user_nm,dance_nm
0,207000097,24696418,1.000,5,0,27,3914,0.999946,0.999983,0.999827,...,-0.986258,0.999988,0.968072,-0.991969,-0.976490,-0.989061,-0.999899,-0.331011,115.001742,1.000000
1,207000097,32507861,1.000,6,2,27,3914,0.999946,0.999983,0.999827,...,-0.986258,0.999988,0.968072,-0.991969,-0.976490,-0.989061,-0.999899,-0.331011,151.376687,1.000000
2,207000097,39060478,1.000,6,2,27,3914,0.999946,0.999983,0.999827,...,-0.986258,0.999988,0.968072,-0.991969,-0.976490,-0.989061,-0.999899,-0.331011,181.889753,1.000000
3,207000097,85909646,1.000,6,2,27,3914,0.999946,0.999983,0.999827,...,-0.986258,0.999988,0.968072,-0.991969,-0.976490,-0.989061,-0.999899,-0.331011,400.048661,1.000000
4,207000097,172874760,1.000,5,0,27,3914,0.999946,0.999983,0.999827,...,-0.986258,0.999988,0.968072,-0.991969,-0.976490,-0.989061,-0.999899,-0.331011,805.012401,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845459,507078823,9873156,0.440,6,0,0,12,0.999862,0.999886,0.999005,...,-0.989222,0.999992,0.945850,-0.997905,-0.995735,-0.999130,-0.994474,0.993043,45.975401,3893.739553
845460,764155292,85274975,0.232,5,0,0,19,0.999687,0.999936,0.999992,...,-0.993992,0.999981,0.948370,-0.999431,-0.997548,-0.999439,-0.977021,0.855312,397.093237,7228.636873
845461,764155292,51463228,0.000,6,0,0,19,0.999687,0.999936,0.999992,...,-0.993992,0.999981,0.948370,-0.999431,-0.997548,-0.999439,-0.977021,0.855312,239.644682,7228.636873
845462,764155292,693496460,0.005,3,0,0,19,0.999687,0.999936,0.999992,...,-0.993992,0.999981,0.948370,-0.999431,-0.997548,-0.999439,-0.977021,0.855312,3229.351326,7228.636873


In [6]:
#划分特征
sparse_cols= ['dance_nm','user_nm','level','sex_em','favorites','play']
dense_cols= [column for column in data][7:-2]

In [7]:
len(dense_cols)

1536

In [8]:
for i in dense_cols:
    print(i)

title_bert0
title_bert1
title_bert2
title_bert3
title_bert4
title_bert5
title_bert6
title_bert7
title_bert8
title_bert9
title_bert10
title_bert11
title_bert12
title_bert13
title_bert14
title_bert15
title_bert16
title_bert17
title_bert18
title_bert19
title_bert20
title_bert21
title_bert22
title_bert23
title_bert24
title_bert25
title_bert26
title_bert27
title_bert28
title_bert29
title_bert30
title_bert31
title_bert32
title_bert33
title_bert34
title_bert35
title_bert36
title_bert37
title_bert38
title_bert39
title_bert40
title_bert41
title_bert42
title_bert43
title_bert44
title_bert45
title_bert46
title_bert47
title_bert48
title_bert49
title_bert50
title_bert51
title_bert52
title_bert53
title_bert54
title_bert55
title_bert56
title_bert57
title_bert58
title_bert59
title_bert60
title_bert61
title_bert62
title_bert63
title_bert64
title_bert65
title_bert66
title_bert67
title_bert68
title_bert69
title_bert70
title_bert71
title_bert72
title_bert73
title_bert74
title_bert75
title_bert76
title_ber

In [9]:
for feat in sparse_cols:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [10]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1,embedding_dim=4)
                       for i,feat in enumerate(sparse_cols)] + [DenseFeat(feat, 1,)
                      for feat in dense_cols]

In [11]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [12]:
#数据构建
train_data = data.sample(frac=1)
train_data.head()

,dance_id,user_id,lable,level,sex_em,favorites,play,title_bert0,title_bert1,title_bert2,...,tag_bert760,tag_bert761,tag_bert762,tag_bert763,tag_bert764,tag_bert765,tag_bert766,tag_bert767,user_nm,dance_nm
642670,849141644,1422000804,0.000,3,0,1,16,0.997254,0.999987,0.779294,...,-0.997498,0.999995,0.979583,-0.999171,-0.983815,-0.996065,-0.999852,0.966526,223806,33785
802095,635019398,314428727,0.007,6,0,0,6,0.999259,0.990914,0.998955,...,-0.979754,0.999882,0.916862,-0.985846,-0.692458,-0.998243,-0.989603,0.928585,134622,22243
203604,975641872,1736086678,1.000,3,1,1946,8016,0.999969,0.999931,0.999994,...,-0.996650,0.999981,0.970229,-0.997671,-0.996071,-0.994866,-0.999925,0.442405,228732,39303
778113,932950116,10233527,0.928,6,0,0,2,0.999869,0.999974,0.993578,...,-0.998542,0.999989,0.856932,-0.999857,-0.894044,-0.998656,-0.999260,0.480240,22440,37099
584540,294422334,2128526357,0.621,3,0,0,15,0.987424,0.999995,0.215316,...,-0.979754,0.999882,0.916862,-0.985846,-0.692458,-0.998243,-0.989603,0.928585,234818,5489


In [13]:
train, test = train_test_split(data, test_size=0.2,random_state=2022)

In [14]:
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [15]:
model = DCN(linear_feature_columns,dnn_feature_columns,task='binary')
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'], )

CrossNet parameterization: vector


In [35]:
history = model.fit(train_model_input, train[['lable']].values,
                    batch_size=1024, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
529/529 [==============================] - 79s 148ms/step - loss: 6.3639 - binary_crossentropy: 6.3639 - val_loss: 6.3567 - val_binary_crossentropy: 6.3566
Epoch 2/20
529/529 [==============================] - 75s 142ms/step - loss: 6.3639 - binary_crossentropy: 6.3639 - val_loss: 6.3567 - val_binary_crossentropy: 6.3566
Epoch 3/20
529/529 [==============================] - 76s 144ms/step - loss: 6.3639 - binary_crossentropy: 6.3639 - val_loss: 6.3567 - val_binary_crossentropy: 6.3566
Epoch 4/20
529/529 [==============================] - 75s 141ms/step - loss: 6.3639 - binary_crossentropy: 6.3639 - val_loss: 6.3567 - val_binary_crossentropy: 6.3566
Epoch 5/20
529/529 [==============================] - 74s 141ms/step - loss: 6.3639 - binary_crossentropy: 6.3638 - val_loss: 6.3567 - val_binary_crossentropy: 6.3566
Epoch 6/20
529/529 [==============================] - 74s 141ms/step - loss: 6.3639 - binary_crossentropy: 6.3638 - val_loss: 6.3567 - val_binary_crossentropy: 6.356

In [15]:
# history = model.fit(train_model_input, train[['lable']].values,
#                     batch_size=256, epochs=100, verbose=2, validation_split=0.2, )
from keras.callbacks import *
# 回调函数
file_path = "DCN_bert.h5"
earlystopping = EarlyStopping(monitor="val_loss", patience=3)
checkpoint = ModelCheckpoint(
    file_path, save_weights_only=False, verbose=1, save_best_only=True)
callbacks_list = [earlystopping, checkpoint]

history = model.fit(train_model_input, train[['lable']].values, 
                   batch_size=256,
                  epochs=100,
                  validation_split=0.2,
                  callbacks=callbacks_list,
                  shuffle=True)
# history=model.fit(train_sparse_x+train_dense_x,
#                   train_label, 
#                   epochs=100,
#                   batch_size=32,
#                   validation_data=(valid_sparse_x+valid_dense_x, valid_label))

Epoch 1/100
2114/2114 [==============================] - 322s 149ms/step - loss: 0.4654 - binary_crossentropy: 0.4602 - val_loss: 0.4451 - val_binary_crossentropy: 0.4363

Epoch 00001: val_loss improved from inf to 0.44514, saving model to deepfm_bert.h5


D:\Anaconda3 2020.11\envs\tf-gpu-25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
2114/2114 [==============================] - 309s 146ms/step - loss: 0.4305 - binary_crossentropy: 0.4178 - val_loss: 0.4479 - val_binary_crossentropy: 0.4337

Epoch 00002: val_loss did not improve from 0.44514
Epoch 3/100
2114/2114 [==============================] - 310s 147ms/step - loss: 0.4079 - binary_crossentropy: 0.3916 - val_loss: 0.4685 - val_binary_crossentropy: 0.4509

Epoch 00003: val_loss did not improve from 0.44514
Epoch 4/100
2114/2114 [==============================] - 309s 146ms/step - loss: 0.3873 - binary_crossentropy: 0.3683 - val_loss: 0.5188 - val_binary_crossentropy: 0.4990

Epoch 00004: val_loss did not improve from 0.44514


In [ ]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [17]:
model.save('DCN_bert.h5')



D:\Anaconda3 2020.11\envs\tf-gpu-25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
